In [3]:
from opentrons.simulate import simulate, format_runlog
from collections import Counter

log, _bundle = simulate(open("../ot2/distribute_2peptide_2_96well.py"))
#print(Counter(format_runlog(log).split(" "))['Picking'])
print(format_runlog(log))


/Users/rathbunc/.opentrons/robot_settings.json not found. Loading defaults
/Users/rathbunc/.opentrons/deck_calibration.json not found. Loading defaults


Amounts of reagents used:  {'peptides': {1: 5120, 2: 5120}, 'frz': 1920}
Transferring 40.0 from A1 of NEST 12 Well Reservoir 15 mL on 7 to A1 of CELLTREAT 96 Well Plate 350 µL on 3
	Picking up tip from A1 of Opentrons 96 Tip Rack 20 µL on 11
	Aspirating 20.0 uL from A1 of NEST 12 Well Reservoir 15 mL on 7 at 7.6 uL/sec
	Dispensing 20.0 uL into A1 of CELLTREAT 96 Well Plate 350 µL on 3 at 7.6 uL/sec
	Aspirating 20.0 uL from A1 of NEST 12 Well Reservoir 15 mL on 7 at 7.6 uL/sec
	Dispensing 20.0 uL into A1 of CELLTREAT 96 Well Plate 350 µL on 3 at 7.6 uL/sec
	Aspirating 20.0 uL from A1 of NEST 12 Well Reservoir 15 mL on 7 at 7.6 uL/sec
	Dispensing 20.0 uL into A2 of CELLTREAT 96 Well Plate 350 µL on 3 at 7.6 uL/sec
	Aspirating 20.0 uL from A1 of NEST 12 Well Reservoir 15 mL on 7 at 7.6 uL/sec
	Dispensing 20.0 uL into A2 of CELLTREAT 96 Well Plate 350 µL on 3 at 7.6 uL/sec
	Aspirating 20.0 uL from A1 of NEST 12 Well Reservoir 15 mL on 7 at 7.6 uL/sec
	Dispensing 20.0 uL into A3 of CELLTREA